In [1]:
import psycopg2
import csv
import os
import sys 
import time
import pandas as pd
assert os.environ.get('DB_PASS') != None , 'empty password!'

In [2]:
conn = psycopg2.connect("dbname='amtrakproject' user='{}' password={}".format(os.environ.get('USER'), os.environ.get('DB_PASS')))
assert conn is not None, 'need to fix conn!!'

### Table_names for reference:
* `station_info`
* `stops`
* `weather_hourly`
* `regional_route`
* to create: `trips`

In [3]:
def execute_command(conn, command):
    try:
        cur = conn.cursor()
        cur.execute(command)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()

### Create track sharing column
* Count of number of other railroads sharing the tracks between stations

In [7]:
track_query = """
              SELECT 
                  CASE 
                      WHEN station_code IN ('KIN', 'WLY', 'MYS') THEN 0
                      WHEN station_code in ('NHV', 'BRP', 'STM', 'NRO') THEN 2
                      ELSE 1
                  END Num_RR_Share
              FROM 
                  stops_joined;
              """

In [8]:
add_tracks_col = """
                 ALTER TABLE stops_joined
                 ADD COLUMN Num_RR_Share integer;
                 """

update_tracks_col = """
                    UPDATE stops_joined
                    SELECT
                        CASE 
                          WHEN station_code IN ('KIN', 'WLY', 'MYS') THEN 0
                          WHEN station_code in ('NHV', 'BRP', 'STM', 'NRO') THEN 2
                          ELSE 1
                        END Num_RR_Share
                    INTO stops_joined
                    FROM stops_joined
                    """

In [11]:
add_shared_tracks = """
                    ALTER TABLE stops_joined
                    ADD COLUMN Num_RR_Share integer;
                    UPDATE stops_joined
                    SET Num_RR_Share = (
                                      SELECT  
                                          CASE 
                                              WHEN station_code IN ('KIN', 'WLY', 'MYS') THEN 0
                                              WHEN station_code in ('NHV', 'BRP', 'STM', 'NRO') THEN 2
                                              ELSE 1
                                          END Num_RR_Share
                                      FROM 
                                          stops_joined
                    );
                    """

In [12]:
execute_command(conn, add_shared_tracks)

more than one row returned by a subquery used as an expression



In [ ]:
def execute_and_get_results(conn, command):
    try:
        cur = conn.cursor()
        cur.execute(command)
        return cur.fetchall()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()